 ![CellphoneDB Logo](https://www.cellphonedb.org/images/cellphonedb_logo_33.png) CellphoneDB

### Check python version

In [ ]:
import pandas as pd
import numpy as np
import sys
import os

pd.set_option('display.max_columns', 100)
# Define our base directory for the analysis
os.chdir('/home/jovyan/RepTract/CELLPHONEDB/')

Checking that environment contains a Python >= 3.8 as required by CellPhoneDB.

In [ ]:
print(sys.version)

___
## Input files
The differential expression method accepts 5 input files (4 mandatory).
- **cpdb_file_path**: (mandatory) path to the database.
- **meta_file_path**: (mandatory) path to the meta file linking cell barcodes to cluster labels.
- **counts_file_path**: (mandatory) paths to normalized counts file (not z-transformed), either in text format or h5ad (recommended).
- **degs_file_path**: (mandatory) path to the DEG file indicating the differentially expressed genes in each cluster. Only differentially expressed genes that are significant should be included.
- **microenvs_file_path** (optional) path to microenvironment file that groups cell types/clusters by microenvironments. When providing a microenvironment file, CellphoneDB will restrict the interactions to those cells within the microenvironment.

Both, `degs_file_path` and `microenvs_file_path` content will depend on the biological question that the researcher wants to answer.


> In **this example** we are studying how cell-cell interactions change between endometrial cells as epithelia and stromal cells differentiate in response to hormones in the three endoemtrial layers (see image above). Therefore, the `degs_file_path` contains only the genes differentially expressed along the epithelials or stromal/fibroblasts to capture the genes that change along their differentiation in response to progesterone.  The `microenvs_file_path` specifies the cells present in each spaitotemporal compartment of the differentiating endometrium. The `meta_file_path` and `counts_file_path` contain all cells that we are interested in.

> CellphoneDB will retrieve all the interactions occurring between epithelials or stromals and any other celltype in the meta/counts file where: (i) all the proteins are expressed in the corresponding cell type and (ii) at least one gene is differentially expressed by an epithelial/stromal subset.

In [ ]:
cpdb_file_path = '/nfs/team292/vl6/FetalReproductiveTract/v5.0.0/cellphonedb.zip' # this is the downloaded database
meta_file_path = '/nfs/team292/vl6/FetalReproductiveTract/CellPhoneDB/Mullerian_and_Wolffian_early/input/meta.tsv'
counts_file_path = '/nfs/team292/vl6/FetalReproductiveTract/CellPhoneDB/Mullerian_and_Wolffian_early/input/counts_normalised.h5ad'
microenvs_file_path = '/nfs/team292/vl6/FetalReproductiveTract/CellPhoneDB/Mullerian_and_Wolffian_early/input/microenvironments.tsv'
degs_file_path = '/nfs/team292/vl6/FetalReproductiveTract/CellPhoneDB/Mullerian_and_Wolffian_early/input/DEGs_upregulated_genes.tsv'
out_path = '/nfs/team292/vl6/FetalReproductiveTract/CellPhoneDB/Mullerian_and_Wolffian_early/output/'

### Inspect input files

<span style="color:green">**1)**</span> The **metadata** file is compossed of two columns:
- **barcode_sample**: this column indicates the barcode of each cell in the experiment.
- **celltype**: this column denotes the cell label assigned.

In [ ]:
metadata = pd.read_csv(meta_file_path, sep = '\t')
metadata.head(3)

<span style="color:green">**2)**</span>  The **counts** files is a h5ad object from scanpy. The dimensions and order of this object must coincide with the dimensions of the metadata file, i.e. must have the same number of cells in both files.

In [ ]:
import anndata

adata = anndata.read_h5ad(counts_file_path)
adata.shape

Check barcodes in metadata and counts are the same.

In [ ]:
list(adata.obs.index).sort() == list(metadata['Cell']).sort()

<span style="color:green">**3)**</span> **Differentially expressed genes** file is a two columns file indicating which genes are up-regulated (or specific) in a cell type. The first column corresponds to the cluster name (these match with those in the metadata file) and the second column the up-regulated gene. The remaining columns are ignored by CellPhoneDB. All genes present in this file will be taken into account, thus the user must provide in this file only those genes considered as up-regulated or relevant for the analysis.

In [ ]:
degs = pd.read_csv(degs_file_path, sep = '\t')
degs.head(3)

<span style="color:green">**4)**</span> **Micronevironments** defines the cell types that belong to a a given microenvironemnt. CellPhoneDB will only calculate interactions between cells that belong to a given microenvironment. In this file we are defining two microenvionments.

In [ ]:
microenv = pd.read_csv(microenvs_file_path, sep = '\t')
microenv

Displaying cells grouped per microenvironment

In [ ]:
microenv.groupby('microenvironment')['celltype'].apply(lambda x : list(x.value_counts().index))

<span style="color:green">**5)**</span> **Check cell type names are the same** 

In [ ]:

# all cells in microenv are in meta
[ item in set(metadata['cell_type']) for item in set(microenv['celltype']) ]

In [ ]:
# all cells in microenv are in meta - who is not?
list(set(microenv['celltype']) - set(metadata['cell_type']) )

In [ ]:
# all cells in degs are in meta
[ item in set(metadata['cell_type']) for item in set(degs['cluster']) ]

In [ ]:
# all cells in degs are in meta - who is not?
list(set(degs['cluster']) - set(metadata['cell_type']) )

____
# Run CellphoneDB with differential analysis (method 3)
The output of this method will be saved in `out_path` and also assigned to the predefined variables.

In [ ]:
from cellphonedb.src.core.methods import cpdb_degs_analysis_method
res = \
    cpdb_degs_analysis_method.call(
        cpdb_file_path = cpdb_file_path, 
        meta_file_path = meta_file_path, 
        counts_file_path = counts_file_path,
        degs_file_path = degs_file_path,
        counts_data = 'hgnc_symbol',
        microenvs_file_path=microenvs_file_path,
        threshold = 0.1,
        result_precision = 3,
        separator = '|',
        debug = False,
        output_path = out_path,
        output_suffix = None,
        score_interactions = True,
        threads = 4)

In [ ]:
res.keys()

___
### Description of output files

**Relevant interaction** fields:
- **id_cp_interaction**: interaction identifier.
- **interacting_pair**: Name of the interacting pairs.
- **partner A/B**: Identifier for the first interacting partner (A) or the second (B). It could be: UniProt (prefix simple:) or complex (prefix complex:)
- **gene A/B**: Gene identifier for the first interacting partner (A) or the second (B).
- **secreted**: True if one of the partners is secreted.
- **receptor A/B**: True if the first interacting partner (A) or the second (B) is annotated as a receptor in our database.
- **annotation_strategy**: Curated if the interaction was annotated by the CellPhoneDB developers. Other value if it was added by the user.
- **is_integrin**: True if one of the partners is integrin.
- **cell_a|cell_b**: 1 if interaction is detected as significant, 0 if not.

In [ ]:
res['relevant_interactions_result']

In [ ]:
res['relevant_interactions_result'].head(3)

**Deconvoluted** fields:
- **gene_name**: Gene identifier for one of the subunits that are participating in the interaction defined in “means.csv” file. The identifier will depend on the input of the user list.
- **uniprot**: UniProt identifier for one of the subunits that are participating in the interaction defined in “means.csv” file.
- **is_complex**: True if the subunit is part of a complex. Single if it is not, complex if it is.
- **protein_name**: Protein name for one of the subunits that are participating in the interaction defined in “means.csv” file.
- **complex_name**: Complex name if the subunit is part of a complex. Empty if not.
- **id_cp_interaction**: Unique CellPhoneDB identifier for each of the interactions stored in the database.
- **mean**: Mean expression of the corresponding gene in each cluster.

In [ ]:
res['deconvoluted_result'].head(3)

**Means** fields:
- **id_cp_interaction**: Unique CellPhoneDB identifier for each interaction stored in the database.
- **interacting_pair**: Name of the interacting pairs.
- **partner A or B**: Identifier for the first interacting partner (A) or the second (B). It could be: UniProt (prefix simple:) or complex (prefix complex:)
- **gene A or B**: Gene identifier for the first interacting partner (A) or the second (B). The identifier will depend on the input user list.
- **secreted**: True if one of the partners is secreted.
- **Receptor A or B**: True if the first interacting partner (A) or the second (B) is annotated as a receptor in our database.
- **annotation_strategy**: Curated if the interaction was annotated by the CellPhoneDB developers. Otherwise, the name of the database where the interaction has been downloaded from.
- **is_integrin**: True if one of the partners is integrin.
- **means**: Mean values for all the interacting partners: mean value refers to the total mean of the individual partner average expression values in the corresponding interacting pairs of cell types. If one of the mean values is 0, then the total mean is set to 0.

In [ ]:
res['means_result'].head(3)

**Significant means** fields:
- **id_cp_interaction**: Unique CellPhoneDB identifier for each interaction stored in the database.
- **interacting_pair**: Name of the interacting pairs.
- **partner A or B**: Identifier for the first interacting partner (A) or the second (B). It could be: UniProt (prefix simple:) or complex (prefix complex:)
- **gene A or B**: Gene identifier for the first interacting partner (A) or the second (B). The identifier will depend on the input user list.
- **secreted**: True if one of the partners is secreted.
- **Receptor A or B**: True if the first interacting partner (A) or the second (B) is annotated as a receptor in our database.
- **annotation_strategy**: Curated if the interaction was annotated by the CellPhoneDB developers. Otherwise, the name of the database where the interaction has been downloaded from.
- **is_integrin**: True if one of the partners is integrin.
- **significant_mean**: Significant mean calculation for all the interacting partners. If the interaction has been found relevant, the value will be the mean. Alternatively, the value is set to 0.

In [ ]:
res['significant_means'].head(3)

In [ ]:
res['interaction_scores'].head(3)

# Explore CellPhoneDB results
This module allows to filter CellPhoneDB results by specifying either
- **cell types pairs**. We can specify two list of cell types (`query_celltype_1`and `query_celltype_2`), the method will subset the interactions to those pairs of cells. Cell types within each list will not be paired.  If we are interested in filtering interactions ocurring between a given cell to all the rest of cells in the dataset we can define `query_celltype_1 = 'All'` and `query_celltype_2 = ['cellA', 'cellB', ...]`. 
- **genes** The argument `genes` allows the user to filter interactions in which a gene participates.
- or **specific interactions** to define specific interactions based on the interaction name `query_interactions`.

This method filters the rows and columns of the significant_means file; NaN value correspond to interacting pairs found not significant, non-NaN value correspond to those interacting pairs found relevant.

In [ ]:
from cellphonedb.utils import search_utils

search_results = search_utils.search_analysis_results(
    query_cell_types_1 = ['WolffianDuct_Epithelium'],  # List of cells 1, will be paired to cells 2 (list or 'All').
    query_cell_types_2 = ['MüllerianDuct_Mesenchyme', 'MüllerianDuct_Epithelium'],     # List of cells 2, will be paired to cells 1 (list or 'All').
#     query_genes = ['LRP5'],                                       # filter interactions based on the genes participating (list).
#     query_interactions = ['CSF1_CSF1R'],                            # filter intereactions based on their name (list).
    significant_means = res['significant_means'],                          # significant_means file generated by CellPhoneDB.
    deconvoluted = res['deconvoluted_result'],                                    # devonvoluted file generated by CellPhoneDB.
    separator = '|',                                                # separator (default: |) employed to split cells (cellA|cellB).
    long_format = True                                              # converts the output into a wide table, removing non-significant interactions
)

# search_results.head(60)
search_results = search_results[search_results['interacting_cells'].isin(['WolffianDuct_Epithelium|MüllerianDuct_Mesenchyme',
                                                                         'WolffianDuct_Epithelium|MüllerianDuct_Epithelium'])]
print(search_results.shape)

search_results.sort_values('significant_mean', ascending=False).head(50)

In [ ]:
from cellphonedb.utils import search_utils

search_results = search_utils.search_analysis_results(
    query_cell_types_1 = ['WolffianDuct_Mesenchyme'],  # List of cells 1, will be paired to cells 2 (list or 'All').
    query_cell_types_2 = ['MüllerianDuct_Mesenchyme', 'MüllerianDuct_Epithelium'],     # List of cells 2, will be paired to cells 1 (list or 'All').
#     query_genes = ['LRP5'],                                       # filter interactions based on the genes participating (list).
#     query_interactions = ['CSF1_CSF1R'],                            # filter intereactions based on their name (list).
    significant_means = res['significant_means'],                          # significant_means file generated by CellPhoneDB.
    deconvoluted = res['deconvoluted_result'],                                    # devonvoluted file generated by CellPhoneDB.
    separator = '|',                                                # separator (default: |) employed to split cells (cellA|cellB).
    long_format = True                                              # converts the output into a wide table, removing non-significant interactions
)

# search_results.head(60)
search_results = search_results[search_results['interacting_cells'].isin(['WolffianDuct_Mesenchyme|MüllerianDuct_Mesenchyme',
                                                                         'WolffianDuct_Mesenchyme|MüllerianDuct_Epithelium'])]
print(search_results.shape)
search_results.sort_values('significant_mean', ascending=False).tail(50)

### Save interactions for supplementary table

In [ ]:
from cellphonedb.utils import search_utils

search_results = search_utils.search_analysis_results(
    query_cell_types_1 = ['WolffianDuct_Mesenchyme', 'WolffianDuct_Epithelium'],  # List of cells 1, will be paired to cells 2 (list or 'All').
    query_cell_types_2 = ['MüllerianDuct_Mesenchyme', 'MüllerianDuct_Epithelium'],     # List of cells 2, will be paired to cells 1 (list or 'All').
#     query_genes = ['LRP5'],                                       # filter interactions based on the genes participating (list).
#     query_interactions = ['CSF1_CSF1R'],                            # filter intereactions based on their name (list).
    significant_means = res['significant_means'],                          # significant_means file generated by CellPhoneDB.
    deconvoluted = res['deconvoluted_result'],                                    # devonvoluted file generated by CellPhoneDB.
    separator = '|',                                                # separator (default: |) employed to split cells (cellA|cellB).
    long_format = True                                              # converts the output into a wide table, removing non-significant interactions
)

# search_results.head(60)
search_results = search_results[search_results['interacting_cells'].isin(['WolffianDuct_Mesenchyme|MüllerianDuct_Mesenchyme',
                                                                         'WolffianDuct_Mesenchyme|MüllerianDuct_Epithelium', 
                                                                         'WolffianDuct_Epithelium|MüllerianDuct_Mesenchyme', 
                                                                         'WolffianDuct_Epithelium|MüllerianDuct_Epithelium'])]
print(search_results.shape)
# search_results.sort_values('significant_mean', ascending=False).tail(50)

In [ ]:
search_results.to_csv('/nfs/team292/vl6/FetalReproductiveTract/CellPhoneDB/Mullerian_and_Wolffian_early/output/wolffian_to_mullerian_signalling.csv')